<img src="https://brand.umich.edu/assets/brand/style-guide/logo-guidelines/U-M_Logo-Horizontal-Hex.png" alt="Drawing" style="width: 300px;" align="left"/><br>
    
## Week 4: Building a learning analytics dashboard

In this week, you will build an interactive dashboard to visualize prediction results and other important information about the course and students. As with last week, you are building this as a prototype, and the specific user persona we have in mind for you to design around is a non-technical instructor who is relatively new to teaching online and has moderate to low data literacy skills. This means that the resulting dashboards should be easily interpretable. Your dashboard should help this instructor identify students who are not likely to succeed (**Part A**). From there, you will bould two dashboards that provide course-level information on assessment (**Part B**) and VLE use (**Part C**). You will then provide a view for each student on key metrics (**Part D**). Lastly, you will put these individual dashboards together into one (**Part E**). 


You will be using the [jupyter-dash extension](https://github.com/plotly/jupyter-dash) to build a [Plotly Dash](https://github.com/plotly/dash) app interactively within Jupyter environments.

Dash is the most downloaded, trusted Python framework for building ML & data science web apps. You can check out the documentation and some tutorials for Plotly Dash [here](https://dash.plotly.com/).

Resources:
* [A medium blog - Introducing JupyterDash](https://medium.com/plotly/introducing-jupyterdash-811f1f57c02e)
* [Plotly Dash documentation](https://dash.plotly.com/)
* [A Youtube tutorial on dash](https://www.youtube.com/watch?v=hSPmj7mK6ng)
* [Dash gallery](https://dash-gallery.plotly.host/Portal/)

The dashboard should enable users to quickly extract insights to answer the following questions:
* Who are the students in my course? (20 pts)
* Which students are likely to drop out or fail my course? (20 pts)
* How do my course compare to other courses in terms of pass_rate and dropout_rate? (20 pts)
* What is the grade distribution of my course? (20 pts)
* Integrate everything into a single dashboard (20 pts)

The dashboard will be manually graded based on the following rubric: TBD

You can also check out [OU Analyse](https://analyse.kmi.open.ac.uk/), screenshot below, as an example of how the authors of the dataset have developed a dashboard for their institution. You can request a demo of the dashboard by enter your email.
![OU Analyse](https://analyse.kmi.open.ac.uk/resources/images/project_info/screenshot_01.png)

# Loading libraries

In [1]:
import pandas as pd
import numpy as np

# Import jupyter dash
from jupyter_dash import JupyterDash
import os
try:
    os.environ.pop('http_proxy')
    os.environ.pop('https_proxy')
except KeyError:
    pass

# Import dash
import dash
from dash import dcc
from dash import html

# Import plotly
import plotly.graph_objs as go
import plotly.express as px

# Set up jupyter proxy
#JupyterDash.infer_jupyter_proxy_config()

# A quick tutorial

In essence, a plotly-dash dashboard consists of 3 components:
* The **dash components** (e.g., dropdown, slider, checklist, etc.). See the documentation [here](https://dash.plotly.com/dash-core-components)
* The **plotly** graphs (e.g., linegraph, scatter plot, heatmap, etc.). See the documentation [here](https://plotly.com/python/)
* The **callback** to connects the dash components to plotly graphs, making it an interactive dashboard. See the documentation [here](https://dash.plotly.com/basic-callbacks) 



In [2]:
# Let's import a sample dataframe
df = pd.read_csv('assets/course_passrate.csv')
df.head(5)

,code_module,code_presentation,pass_rate,dropout_rate
0,AAA,2013J,0.860681,0.156658
1,AAA,2014J,0.846154,0.180822
2,BBB,2013B,0.636292,0.285795
3,BBB,2013J,0.672944,0.287886
4,BBB,2014B,0.647373,0.303782


## Step 1. The Dash components (i.e. layouts)

In [3]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

# Create server variable with Flask server object for use with gunicorn
server = app.server

# Create a unique list of code_module
available_indicators1 = df['code_module'].unique()

# Step 1

app.layout = html.Div([
            # Create a html title for the dashboard
            html.H1("This is the title"),
    
            # Create a graph, we will configure the graph using plotly express in step 3 
            dcc.Graph(id='graph-with-dropdown'),
    
            # Create a dropdown menu based on code_module
            dcc.Dropdown(
                id='crossfilter-xaxis-column',
                options=[{'label': i, 'value': i} for i in available_indicators1],
                value='AAA' # the default is code_module AAA
            )])

# Run the app
app.run_server(mode="inline", port = 8100)

# You will see we have the dropdown menu but nothing happens yet

## Step 2 & 3. Callback and create plotly graph

In [4]:
# Let's import a sample dataframe
df = pd.read_csv('assets/course_passrate.csv')

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

# Create server variable with Flask server object for use with gunicorn
server = app.server

# Create a unique list of code_module
available_indicators1 = df['code_module'].unique()

# Step 1

app.layout = html.Div([
            # Create a html title for the dashboard
            html.H1("This is a bar chart"),
    
            # Create a graph, we will configure the graph using plotly express in step 3 
            dcc.Graph(id='graph-with-dropdown'),
    
            # Create a dropdown menu based on code_module
            dcc.Dropdown(
                id='crossfilter-xaxis-column',
                options=[{'label': i, 'value': i} for i in available_indicators1],
                value='AAA' # the default is code_module AAA
            )])

# Step 2
# Callback using input from dropdown menu to generate graph
# You can have multiple inputs and multiple outputs
@app.callback(
    dash.dependencies.Output('graph-with-dropdown', 'figure'),
    [dash.dependencies.Input('crossfilter-xaxis-column', 'value')])

# Step 3
# Define the graph with plotly express
def update_figure(code_module):
    filtered_df = df[(df.code_module == code_module)]
    figure = px.bar(filtered_df, x='code_presentation', y='pass_rate')
    return figure # You must return all the output(s) in step 2
    
# Run the app
app.run_server(mode="inline", port = 8101)

## Let's plot a scatter plot for pass_rate and dropout_rate

In [5]:
# Let's import a sample dataframe
df = pd.read_csv('assets/course_passrate.csv')

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

# Create server variable with Flask server object for use with gunicorn
server = app.server

# Create a unique list of code_presentation
available_indicators1 = df['code_presentation'].unique()

# Step 1
app.layout = html.Div([
            # Create a html title for the dashboard
            html.H1("This is a scatter plot"),
    
            # Create a graph, we will configure the graph using plotly express in step 3 
            dcc.Graph(id='graph'),
    
            # Create a checklist based on code_presentation
            dcc.Checklist(
                    id = 'checklist',
                    options=[{'label': i, 'value': i} for i in available_indicators1],
                    value=available_indicators1 # Default values contain all code_presentation
            )
])

# Step 2
# Callback using inputs from the checklist to generate the graph
@app.callback(
    dash.dependencies.Output('graph', 'figure'),
    [dash.dependencies.Input('checklist', 'value')])

# Step 3
# Define the graph with plotly express
def update_figure(code_presentation):
    filtered_df = df[(df.code_presentation.isin(code_presentation))]
    figure = px.scatter(filtered_df, 
                     x="pass_rate", 
                     y="dropout_rate", 
                     color="code_presentation",
                     hover_name='code_module')
    return figure # You must return all the output(s) in step 2
    
# Run the app
app.run_server(mode="inline", port = 8102)

# Build the demo app

In [6]:
df = pd.read_csv('assets/country_indicators.csv')
available_indicators = df['Indicator Name'].unique()
df

,Country Name,Indicator Name,Year,Value
0,Arab World,"Agriculture, value added (% of GDP)",1962,NaN
1,Arab World,CO2 emissions (metric tons per capita),1962,0.760996
2,Arab World,Domestic credit provided by financial sector (...,1962,18.168690
3,Arab World,Electric power consumption (kWh per capita),1962,NaN
4,Arab World,Energy use (kg of oil equivalent per capita),1962,NaN
...,...,...,...,...
36955,Zimbabwe,"Industry, value added (% of GDP)",2007,33.074953
36956,Zimbabwe,"Inflation, GDP deflator (annual %)",2007,0.894887
36957,Zimbabwe,"Life expectancy at birth, total (years)",2007,44.177756
36958,Zimbabwe,Population density (people per sq. km of land ...,2007,34.374559


In [7]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

# Create server variable with Flask server object for use with gunicorn
server = app.server

# Design app layout
app.layout = html.Div([
    html.Div([

        html.Div([
            # Create a dropdown menu on x-axis
            dcc.Dropdown(
                id='crossfilter-xaxis-column',
                options=[{'label': i, 'value': i} for i in available_indicators],
                value='Fertility rate, total (births per woman)'
            ),
            
            # Create a multiple choice menu on x-axis
            dcc.RadioItems(
                id='crossfilter-xaxis-type',
                options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
                value='Linear',
                labelStyle={'display': 'inline-block'}
            )
        ],
        style={'width': '49%', 'display': 'inline-block'}),

        html.Div([
            # Create a dropdown menu on y-axis
            dcc.Dropdown(
                id='crossfilter-yaxis-column',
                options=[{'label': i, 'value': i} for i in available_indicators],
                value='Life expectancy at birth, total (years)'
            ),
            # Create a multiple choice menu on y-axis
            dcc.RadioItems(
                id='crossfilter-yaxis-type',
                options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
                value='Linear',
                labelStyle={'display': 'inline-block'}
            )
        ], style={'width': '49%', 'float': 'right', 'display': 'inline-block'})
    ], style={
        'borderBottom': 'thin lightgrey solid',
        'backgroundColor': 'rgb(250, 250, 250)',
        'padding': '10px 5px'
    }),
    
    # Create a dashboard which consists of a scatter plot, x-time-series, and y-time-series
    html.Div([
        dcc.Graph(
            id='crossfilter-indicator-scatter',
            hoverData={'points': [{'customdata': 'Japan'}]}
        )
    ], style={'width': '49%', 'display': 'inline-block', 'padding': '0 20'}),
    html.Div([
        dcc.Graph(id='x-time-series'),
        dcc.Graph(id='y-time-series'),
    ], style={'display': 'inline-block', 'width': '49%'}),
    
    # Create a slider on year
    html.Div(dcc.Slider(
        id='crossfilter-year--slider',
        min=df['Year'].min(),
        max=df['Year'].max(),
        value=df['Year'].max(),
        marks={str(year): str(year) for year in df['Year'].unique()},
        step=None
    ), style={'width': '49%', 'padding': '0px 20px 20px 20px'})
])

# Call back function
# It will take the 5 inputs and produce 1 output which is the scatter plot figure
@app.callback(
    dash.dependencies.Output('crossfilter-indicator-scatter', 'figure'),
    [dash.dependencies.Input('crossfilter-xaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-yaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-xaxis-type', 'value'),
     dash.dependencies.Input('crossfilter-yaxis-type', 'value'),
     dash.dependencies.Input('crossfilter-year--slider', 'value')])

# Define update_graph function
# It basically filters the df based on users' input (e.g. dropdown menu, slider, multiple choice)
def update_graph(xaxis_column_name, yaxis_column_name,
                 xaxis_type, yaxis_type,
                 year_value):
    dff = df[df['Year'] == year_value]

    return {
        'data': [dict(
            x=dff[dff['Indicator Name'] == xaxis_column_name]['Value'],
            y=dff[dff['Indicator Name'] == yaxis_column_name]['Value'],
            text=dff[dff['Indicator Name'] == yaxis_column_name]['Country Name'],
            customdata=dff[dff['Indicator Name'] == yaxis_column_name]['Country Name'],
            mode='markers',
            marker={
                'size': 25,
                'opacity': 0.7,
                'color': 'orange',
                'line': {'width': 2, 'color': 'purple'}
            }
        )],
        'layout': dict(
            xaxis={
                'title': xaxis_column_name,
                'type': 'linear' if xaxis_type == 'Linear' else 'log'
            },
            yaxis={
                'title': yaxis_column_name,
                'type': 'linear' if yaxis_type == 'Linear' else 'log'
            },
            margin={'l': 40, 'b': 30, 't': 10, 'r': 0},
            height=450,
            hovermode='closest'
        )
    }


def create_time_series(dff, axis_type, title):
    return {
        'data': [dict(
            x=dff['Year'],
            y=dff['Value'],
            mode='lines+markers'
        )],
        'layout': {
            'height': 225,
            'margin': {'l': 20, 'b': 30, 'r': 10, 't': 10},
            'annotations': [{
                'x': 0, 'y': 0.85, 'xanchor': 'left', 'yanchor': 'bottom',
                'xref': 'paper', 'yref': 'paper', 'showarrow': False,
                'align': 'left', 'bgcolor': 'rgba(255, 255, 255, 0.5)',
                'text': title
            }],
            'yaxis': {'type': 'linear' if axis_type == 'Linear' else 'log'},
            'xaxis': {'showgrid': False}
        }
    }

# Call back for time-series graph on x-axis
@app.callback(
    dash.dependencies.Output('x-time-series', 'figure'),
    [dash.dependencies.Input('crossfilter-indicator-scatter', 'hoverData'),
     dash.dependencies.Input('crossfilter-xaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-xaxis-type', 'value')])
def update_y_timeseries(hoverData, xaxis_column_name, axis_type):
    country_name = hoverData['points'][0]['customdata']
    dff = df[df['Country Name'] == country_name]
    dff = dff[dff['Indicator Name'] == xaxis_column_name]
    title = '<b>{}</b><br>{}'.format(country_name, xaxis_column_name)
    return create_time_series(dff, axis_type, title)

# Call back for time-series graph on y-axis
@app.callback(
    dash.dependencies.Output('y-time-series', 'figure'),
    [dash.dependencies.Input('crossfilter-indicator-scatter', 'hoverData'),
     dash.dependencies.Input('crossfilter-yaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-yaxis-type', 'value')])
def update_x_timeseries(hoverData, yaxis_column_name, axis_type):
    dff = df[df['Country Name'] == hoverData['points'][0]['customdata']]
    dff = dff[dff['Indicator Name'] == yaxis_column_name]
    return create_time_series(dff, axis_type, yaxis_column_name)

In [8]:
# Run the app inside jupyter notebook
app.run_server(mode="inline", port =8051)


# How the grading works

After finishing the assignment, you will need to generate a sharable link to this notebook on Coursera and send the link to us - the instruction team. We will then re-run your notebook to reproduce the dashboard.

**Please make sure your notebook is reproducible!** 

Once you are finished with the assignment, make sure to `restart the kernel` and re-run the cells to check if the notebook can be reproduced.

---

## Part A (20 points) 

Create a dashboard that shows the a list of students and their associated prediction for pass/fail the course. The dashboard should allow instructors to filter by code_module and code_presentation     
You can use the predicted probabilities and predicted label for students in 'assets/student_pred.csv'

In [9]:
# Read file: 
df_stu = pd.read_csv("assets/student_pred.csv")

# Pass/Fail 
df_stu['pred_string'] = np.where(df_stu['pred'] == 0, 'fail', 'pass')

# Server:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

# Create server variable with Flask server object for use with gunicorn
server = app.server


# Step 1: Create a layout (title, dropdown menu, slider, etc...)
app.layout = html.Div([
            # Create a html title for the dashboard
            html.H2("List of Students Predicted to Pass/Fail"),
                
            # Create a dropdown: code_presentation
            html.Div([dcc.Dropdown(
                    id = 'code_pres',
                    options=[{'label': i, 'value': i} for i in df_stu['code_presentation'].unique()],
                    value='2013J',
                    placeholder='Select code_presentation'# Default values contain all code_presentation
            
            )], style={'width': '48%', 'display': 'inline-block'}),
            
            # Create a dropdown: code_module
            html.Div([dcc.Dropdown(
                id = 'code_module',
                options=[{'label': i, 'value': i} for i in df_stu['code_module'].unique()],
                value='AAA',
                placeholder='Select code_module')], # Default values contain all code_presentation
            style={'width': '48%', 'display': 'inline-block'}),
    
        
            # Create radio button: 
            html.Div([
            dcc.RadioItems(id = 'pass_fail', 
                           options= [{'label': k, 'value': k} for k in ['pass','fail']],
                           value = 'pass', 
                           labelStyle={'display': 'inline-block'})], 
                           
                           ),
    
            
            # Create a table, we will configure the graph using plotly express in step 3 
            dcc.Graph(id='graph_students_pass')
            

    
])


# Step 2: Callback to connect input(s) to output(s)
@app.callback(
    dash.dependencies.Output('graph_students_pass', 'figure'),
    [dash.dependencies.Input('code_module', 'value'),
     dash.dependencies.Input('code_pres', 'value'),
     dash.dependencies.Input('pass_fail', 'value')]) 

def update_figure(code_module, code_pres, pass_fail):
                                     
    df_filter = df_stu[df_stu.pred_string == pass_fail]
    df_result = df_filter[(df_filter['code_module'] == code_module) & 
                                            (df_filter['code_presentation'] == code_pres)]
    
    # Table: list of students pass: 
    fig_students = go.Table(
        header=dict(values=['id_student', 'pred_proba', 'prediction (pass/fail)']),
        cells=dict(values=[df_result['id_student'], 
                           df_result['pred_proba'].map('{:,.3f}'.format), 
                           df_result['pred_string']]))

    data = [fig_students]
    fig_table = go.Figure(data)
    
    
 
    return fig_table
    
# Un-comment to run the app
app.run_server(mode="inline", port =8052)

---

## Part B (20 points)

Create a dashboard that shows the submission rate of each of the first 3 TMAs and the grade distribution for each TMA. The dashboard should allow instructors to filter by code_module and code_presentation  

You can use the pass_rate and submission_rate in 'assets/pass_rate.csv'

In [10]:
# Prepare data: 
 
assessments = pd.read_csv('assets/assessments.csv')
 
# Filter out TMA ('Tutor Marked Assessments')
assessments = assessments[assessments['assessment_type'] == 'TMA']
 
# Sort dates: 
def smallest3(data):
    d = {}
    sorted_date = np.sort(data['date'])
    d['s1'] = sorted_date[0]
    d['s2'] = sorted_date[1]
    d['s3'] = sorted_date[2]
    return pd.Series(d)
 
# Get sorted dates table:
TMA_dates = assessments.groupby(["code_module","code_presentation"]).apply(smallest3).reset_index()
 
# Merge assessments with TMA_dates: 
ass_TMA = assessments.merge(TMA_dates, on = ['code_module', 'code_presentation'])
 
# Drop rows which have assessment dates not in s1 or s2 or s3 
ass_TMA = ass_TMA[np.logical_or(np.logical_or(ass_TMA['date'] == ass_TMA['s1'], ass_TMA['date'] == ass_TMA['s2']),
                    ass_TMA['date'] == ass_TMA['s3'])]
 
# Read pass_rate csv:
pass_rate = pd.read_csv("assets/pass_rate.csv")
 
# merge pass_rate with ass_TMA
pass_TMA = ass_TMA.merge(pass_rate, on = ['code_module', 'code_presentation', 'id_assessment'])
 
def tma1_sub(x):
    return x['submission_rate'] if x['date'] == x['s1'] else np.nan
 
def tma2_sub(x):
    return x['submission_rate'] if x['date'] == x['s2'] else np.nan
 
def tma3_sub(x):
    return x['submission_rate'] if x['date'] == x['s3'] else np.nan
 
pass_TMA["TMA1_sub"] = pass_TMA.apply(tma1_sub, axis=1)
pass_TMA["TMA2_sub"] = pass_TMA.apply(tma2_sub, axis=1)
pass_TMA["TMA3_sub"] = pass_TMA.apply(tma3_sub, axis=1)
  
    
    
def tma1_pass(x):
    return x['pass_rate'] if x['date'] == x['s1'] else np.nan
 
def tma2_pass(x):
    return x['pass_rate'] if x['date'] == x['s2'] else np.nan
 
def tma3_pass(x):
    return x['pass_rate'] if x['date'] == x['s3'] else np.nan
 
pass_TMA["TMA1_pass"] = pass_TMA.apply(tma1_pass, axis=1)
pass_TMA["TMA2_pass"] = pass_TMA.apply(tma2_pass, axis=1)
pass_TMA["TMA3_pass"] = pass_TMA.apply(tma3_pass, axis=1)
 
pass_TMA = pass_TMA.fillna(0)
 
# Group by to get maximum pass_rate & submission_rate instead 
df = pass_TMA.groupby(['code_module', 'code_presentation']).agg(
                {'TMA1_sub': 'max','TMA2_sub': 'max', 'TMA3_sub': 'max', 
                 'TMA1_pass': 'max','TMA2_pass': 'max', 'TMA3_pass': 'max'}).reset_index()
# View data:
#df.head() 


# Server:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
 
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)
 
# Create server variable with Flask server object for use with gunicorn
server = app.server

 
# Step 1: Create a layout (title, dropdown menu, slider, etc...)
app.layout = html.Div([
            # Create a html title for the dashboard
            html.H2("Submission Rate and Pass Rate"),
            
            # Create a dropdown: code_presentation
            html.Div([dcc.Dropdown(
                    id = 'code_pres',
                    options=[{'label': i, 'value': i} for i in df['code_presentation'].unique()],
                    value='2013J',
                    placeholder='Select code_presentation'# Default values contain all code_presentation
            
            )], style={'width': '48%', 'display': 'inline-block'}),
            
            # Create a dropdown: code_module
            html.Div([dcc.Dropdown(
                id = 'code_module',
                options=[{'label': i, 'value': i} for i in df['code_module'].unique()],
                value='AAA',
                placeholder='Select code_module')], # Default values contain all code_presentation
            style={'width': '48%', 'display': 'inline-block'}),
            
            html.Div([
                # Create a graph for submission rate
                html.Div([dcc.Graph(id='graph_sub')], style={'width': '48%', 'display': 'inline-block'}),
 
 
                # Create another graph for pass rate 
                html.Div([dcc.Graph(id='graph_pass')], style={'width': '48%', 'display': 'inline-block'})
            ])
            
])
 
# Step 2: Callback to connect input(s) to output(s)
@app.callback(
    [dash.dependencies.Output('graph_sub', 'figure'),
     dash.dependencies.Output('graph_pass', 'figure')],
    [dash.dependencies.Input('code_pres', 'value'),
     dash.dependencies.Input('code_module', 'value')])

def update_figure(code_pres, code_module):
    
    # Filter by course: 
    df_filter = df[np.array(df['code_module'] == code_module) & np.array(df['code_presentation'] == code_pres)]
    
    # Return empty dataframe:
    if df_filter.empty:
        fig_sub, fig_pass = {}, {}
    
    else: 
        # Get TMA_sub data and build dataframe: 
        sub_data = df_filter.iloc[0,2:5].to_dict()
        data = [d for d in sub_data.items()]
        df_sub = pd.DataFrame(data, columns=['TMA', 'submission_rate'])

        # Get TMA_pass data and build dataframe: 
        pass_data = df_filter.iloc[0,5:8].to_dict()
        data = [d for d in pass_data.items()]
        df_pass = pd.DataFrame(data, columns=['TMA', 'pass_rate'])

        # Plot submission rate and pass rate bar chart:
        fig_sub = (px.bar(df_sub, x = 'TMA',
                            y = 'submission_rate',
                            title="Submission Rate"))

        fig_pass = (px.bar(df_pass, x = 'TMA',
                            y = 'pass_rate',
                            title="Pass Rate"))
    
    return [fig_sub, fig_pass]
    
# Un-comment to run the app
app.run_server(mode="inline", port =8053)

---

## Part C (20 points)

Create a dashboard that shows the weekly sum click on VLE. It show allows instructors to filter by code_module, code_presentation, VLE_activity_type (e.g., homepage, forums, etc.)

In [11]:
# I have estimated weekly clicks by course and activity type for all students (not per student)

In [12]:
import math
 
# Read files:
stdvle = pd.read_csv('assets/studentVle.csv')
# stdinfo = pd.read_csv("assets/studentInfo.csv")
vle = pd.read_csv("assets/vle.csv")
 
# convert date to week value; create a new column: week
# stdvle_grp["week"] = np.floor(stdvle_grp["date"]/7)
stdvle = stdvle.assign(week = lambda x: x['date'].map(lambda d: math.floor(d/7)))
 
# merge stdvle & vle to get activity_type
df_vle = stdvle.merge(vle, on = ["code_module","code_presentation", "id_site"])
 
# Groupby to get sum of weekly clicks 
 
vle_grp = df_vle.groupby(['code_module', 'code_presentation', 
                          'activity_type', 'week']).agg({'sum_click':'sum'}).reset_index()
 
# Step 1: Create a layout (title, dropdown menu, slider, etc...)
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
 
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)
 
# Create server variable with Flask server object for use with gunicorn
server = app.server
 
# Step 1: Create a layout (title, dropdown menu, slider, etc...)
app.layout = html.Div([
            # Create a html title for the dashboard
            html.H2("Weekly sum of clicks by Course & VLE"),
        
            # Create a dropdown: code_presentation
            html.Div([dcc.Dropdown(
                    id = 'code_pres',
                    options=[{'label': i, 'value': i} for i in vle_grp['code_presentation'].unique()],
                    value='2013J',
                    placeholder='Select code_presentation'# Default values contain all code_presentation
            
            )], style={'width': '31%', 'display': 'inline-block'}),
            
            # Create a dropdown: code_module
            html.Div([dcc.Dropdown(
                id = 'code_module',
                options=[{'label': i, 'value': i} for i in vle_grp['code_module'].unique()],
                value='AAA',
                placeholder='Select code_module')], # Default values contain all code_presentation
            style={'width': '31%', 'display': 'inline-block'}),
            
    
            # Create dropdown: activity_type
            html.Div([dcc.Dropdown(id = 'activity_type', 
                           options= [{'label': i, 'value': i} for i in vle_grp['activity_type'].unique()],
                           value = 'forumng', 
                           placeholder='Select Activity Type')],
            style={'width': '31%', 'display': 'inline-block'}),         
            
                
            # Create a graph, we will configure the graph using plotly express in step 3 
            dcc.Graph(id= 'graph_vle')
           
])
 
# Step 2: Callback to connect input(s) to output(s)
@app.callback(
     dash.dependencies.Output('graph_vle', 'figure'),
    [dash.dependencies.Input('code_pres', 'value'),
     dash.dependencies.Input('code_module', 'value'),
     dash.dependencies.Input('activity_type', 'value')])


def update_figure(code_pres, code_module, act_type):
    
    df_filter = vle_grp[np.array(vle_grp['code_module'] == code_module) & 
                   np.array(vle_grp['code_presentation'] == code_pres) &
                   np.array(vle_grp['activity_type'] == act_type)]
    
    if df_filter.empty:
        fig_vle = {}
 
    else: 
        # Plot submission rate and pass rate
        fig_vle = (px.bar(df_filter, x = 'week',
                            y = 'sum_click'))

    return fig_vle

# Un-comment to run the app
app.run_server(mode="inline", port =8054)

---

## Part D (20 points)

Create a dashboard that shows for each student: 
* Their prediction
* Their grade of the first TMA assignments
* Their weekly VLE activities

It should allow instructors to filter by id_student

In [13]:
import warnings
warnings.filterwarnings('ignore')

# Prepare data: 
# Get df_stud:Defined above in part A:
df_stu

# Pass/Fail 
#df_stu['pred'] = np.where(df_stu['pred'] == 0, 'fail', 'pass')


# Get TMA1 score: Modify function from Assignment 2:
def answer_b():
    
    # YOUR CODE HERE
    # Open assessments table: 
    assessments = pd.read_csv('assets/assessments.csv')
    stdgrade = pd.read_csv('assets/studentAssessment.csv')

    # Filter out TMA ('Tutor Marked Assessments')
    assessments = assessments[assessments['assessment_type']=='TMA']
     
    # Merge: assessments with studentAssessments
    ass_grade = assessments.merge(stdgrade, on = 'id_assessment', how = 'left')

    # Calculate weighted score:
    ass_grade['weighted_score'] = (ass_grade['weight'] * ass_grade['score'])/100

    # But this does not include students who did not submit their TMA. 
    #ass_grade.head()
    
    # Sort dates: 
    def smallest3(data):
        d = {}
        sorted_date = np.sort(data['date'])
        d['s1'] = sorted_date[0]
        d['s2'] = sorted_date[1]
        d['s3'] = sorted_date[2]
        return pd.Series(d)
    
    # Get sorted dates table:
    TMA_dates = assessments.groupby(["code_module","code_presentation"]).apply(smallest3).reset_index()
    
    # Merge assessments with TMA_dates: 
    ass_TMA = assessments.merge(TMA_dates, on = ['code_module', 'code_presentation'], how = 'left')

    # Drop rows which have assessment dates not in s1 or s2 or s3 
    ass_TMA[np.logical_or(np.logical_or(ass_TMA['date'] == ass_TMA['s1'], ass_TMA['date'] == ass_TMA['s2']),
              ass_TMA['date'] == ass_TMA['s3'])]
    
    # Merge: assessments with studentAssessments
    ass_grade = ass_TMA.merge(stdgrade, on = 'id_assessment', how='inner')

    # Calculate weighted score:
    ass_grade['weighted_score'] = (ass_grade['weight'] * ass_grade['score'])/100
    
    # Let's see which students did not submit TMA score: 
    # merge again with df_info: 
    stdinfo = pd.read_csv('assets/studentInfo.csv')
    # Remove withdrawn students: 
    df_info = stdinfo[stdinfo['final_result']!='Withdrawn']
    
    
    ass_grade_info = df_info.merge(ass_grade, on = ['code_module', 'code_presentation', 'id_student'], how = 'left')
    # select columns of interest:
    cols = ['code_module', 'code_presentation', 'id_student', 'date', 'weighted_score', 's1', 's2', 's3']
    select = ass_grade_info[cols]

    def tma1(x):
        return x['weighted_score'] if x['date'] == x['s1'] else np.nan

    def tma2(x):
        return x['weighted_score'] if x['date'] == x['s2'] else np.nan

    def tma3(x):
        return x['weighted_score'] if x['date'] == x['s3'] else np.nan


    select["TMA1"] = select.apply(tma1, axis=1)
    select["TMA2"] = select.apply(tma2, axis=1)
    select["TMA3"] = select.apply(tma3, axis=1)
    select = select.fillna(0)
 
    # Group by to get maximum weighted score instead 
    result = select.groupby(['code_module', 'code_presentation', 'id_student']).agg({'TMA1': 'max','TMA2': 'max', 'TMA3': 'max'}).reset_index()
 
    return result
tma = answer_b()
 
tma_first = tma[['code_module', 'code_presentation', 'id_student', 'TMA1']]

# Modify vle_grp from part C to get weely sum_click per student
vle_grp_student = df_vle.groupby(['id_student','code_module', 'code_presentation', 'week']).agg({'sum_click':'sum'}).reset_index()

# Merge tables: 
df_multi = (tma_first.merge(vle_grp_student,
                 on = ['code_module', 'code_presentation', 'id_student'])
 .merge(df_stu, on = ['code_module', 'code_presentation', 'id_student']))

# Build table: 
table = df_multi[['code_module', 'code_presentation', 'id_student', 'TMA1', 'week', 'sum_click', 'pred']]

# Groupby (Commented out): This groupby did not reduce any rows, only sorted the results
#table_grp = table.groupby(['id_student','code_module', 'code_presentation', 'week']).agg(
    #{'sum_click': 'sum','TMA1': 'max', 'pred': 'first'}).reset_index()

table

# Step 1: Create a layout (title, dropdown menu, slider, etc...)
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
 
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)
 
# Create server variable with Flask server object for use with gunicorn
server = app.server
 
# Step 1: Create a layout (title, dropdown menu, slider, etc...)
app.layout = html.Div([
            # Create a html title for the dashboard
            html.H2("Student Info"),
        
            # Create a dropdown: code_presentation
            html.Div([dcc.Dropdown(
                    id = 'code_pres',
                    options=[{'label': i, 'value': i} for i in table['code_presentation'].unique()],
                    value='2013J',
                    placeholder='Select code_presentation'# Default values contain all code_presentation
            
            )], style={'width': '31%', 'display': 'inline-block'}),
            
            # Create a dropdown: code_module
            html.Div([dcc.Dropdown(
                id = 'code_module',
                options=[{'label': i, 'value': i} for i in table['code_module'].unique()],
                value='AAA',
                placeholder='Select code_module')], # Default values contain all code_presentation
            style={'width': '31%', 'display': 'inline-block'}),
            
    
            # Create dropdown: student_id
            html.Div([dcc.Dropdown(id='id_student', 
                           options= [{'label': i, 'value': i} for i in table['id_student'].unique()],
                           value = 11391, 
                           placeholder='Select Student')],
            style={'width': '31%', 'display': 'inline-block'}),         
            
            html.Div([
            html.Div(id='pred_div', style={'width': '48%', 'display': 'inline-block'}),
            html.Div(id='tma_div', style={'width': '48%', 'display': 'inline-block'})
            ], style={'margin':'10px'}),
                
            # Create a graph, we will configure the graph using plotly express in step 3 
            dcc.Graph(id='graph_std')
           
])
 
# Step 2: Callback to connect input(s) to output(s)
@app.callback(
     [dash.dependencies.Output('graph_std', 'figure'),
      dash.dependencies.Output('pred_div', 'children'),
      dash.dependencies.Output('tma_div', 'children')],
    [dash.dependencies.Input('code_pres', 'value'),
     dash.dependencies.Input('code_module', 'value'),
     dash.dependencies.Input('id_student', 'value')])

def update_figure(code_pres, code_module, id_student):
    
    df_filter = table[np.array(table['code_module'] == code_module) & 
                   np.array(table['code_presentation'] == code_pres) &
                   np.array(table['id_student'] == id_student)]
    
    if df_filter.empty:
        fig_last, pass_fail, tma1_score = {}, '', ''
    
    else: 
 
        # Plot submission rate and pass rate
        fig_last = px.bar(df_filter, x = 'week', y = 'sum_click')

        df_temp = df_filter[['code_module', 'code_presentation', 'id_student', 'TMA1', 'pred']].drop_duplicates()

        if df_temp.iloc[0]['pred'] == 0:
            pass_fail = html.Div([html.Span("Prediction: "), html.Span("FAIL", style={'color':'red'})])
        else:
            pass_fail = html.Div([html.Span("Prediction: "), html.Span("PASS", style={'color':'green'})])

        tma1_score = html.Div("TMA1 weighted score = " + str(df_temp.iloc[0]['TMA1']), 
                                          style={'margin-left':'auto', 'margin-right':'20px', 'text-align': 'right'})

    return [fig_last, pass_fail, tma1_score]

# Un-comment to run the app
app.run_server(mode="inline", port =8055)


---

## Part E (20 points)

Intergrate everything into a single dashboard. It must allows for cross filtering (e.g, all the figures are updated as user select an input). Imagine an instructor will look at part A to pick out students who are at-risk, and then drill down to individual students in part D.

In [14]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

# Create server variable with Flask server object for use with gunicorn
server = app.server


# Step 1: Create a layout (title, dropdown menu, slider, etc...)

app.layout = html.Div([
            # Create a html title for the dashboard
            html.H2("Open University Learning Analytics Dashboard"),
                
            # Create a dropdown: code_presentation
            html.Div([dcc.Dropdown(
                    id = 'code_pres',
                    options=[{'label': i, 'value': i} for i in df_stu['code_presentation'].unique()],
                    value='2013J',
                    placeholder='Select code_presentation'# Default values contain all code_presentation
            
            )], style={'width': '48%', 'display': 'inline-block'}),
            
            # Create a dropdown: code_module
            html.Div([dcc.Dropdown(
                id = 'code_module',
                options=[{'label': i, 'value': i} for i in df_stu['code_module'].unique()],
                value='AAA',
                placeholder='Select code_module')], # Default values contain all code_presentation
            style={'width': '48%', 'display': 'inline-block'}),
    
            # Add Part A:
            html.H3("Part A: Student Predictions"),
             # Create radio button: 
            html.Div([
            dcc.RadioItems(id = 'pass_fail', 
                           options= [{'label': k, 'value': k} for k in ['pass','fail']],
                           value = 'pass', 
                           labelStyle={'display': 'inline-block'})],
                           
                           ),
    

            # Create a table, we will configure the graph using plotly express in step 3 
            dcc.Graph(id='graph_students_pass'),
            
            # Add Part B: 
            html.H3("Part B: Submission Rate and Pass Rate"),
            
            html.Div([
                # Create a graph for submission rate
                html.Div([dcc.Graph(id='graph_sub')], style={'width': '48%', 'display': 'inline-block'}),


                # Create another graph for pass rate 
                html.Div([dcc.Graph(id='graph_pass')], style={'width': '48%', 'display': 'inline-block'})
            ]),
    
            # Add Part C:    
            html.H3("Part C: Weekly sum of clicks by Course & VLE"),
            
            # Create dropdown: activity_type
            html.Div([dcc.Dropdown(id = 'activity_type', 
                           options= [{'label': i, 'value': i} for i in vle_grp['activity_type'].unique()],
                           value = 'forumng', 
                           placeholder='Select Activity Type')],
            style={'width': '31%', 'display': 'inline-block'}),         
            
                
            # Create a graph, we will configure the graph using plotly express in step 3 
            dcc.Graph(id= 'graph_vle'),
    
    
            # Add Part D:
            html.H3("Part D: Student Info"),
    
            # Create dropdown: student_id
            html.Div([dcc.Dropdown(id='id_student', 
                           options= [{'label': i, 'value': i} for i in table['id_student'].unique()],
                           value = 11391, 
                           placeholder='Select Student')],
            style={'width': '31%', 'display': 'inline-block'}),         
            
            html.Div([
            html.Div(id='pred_div', style={'width': '48%', 'display': 'inline-block'}),
            html.Div(id='tma_div', style={'width': '48%', 'display': 'inline-block'})
            ], style={'margin':'10px'}),
                
            # Create a graph, we will configure the graph using plotly express in step 3 
            dcc.Graph(id='graph_std')
            
])


# Step 2: Callback to connect input(s) to output(s)
@app.callback(
    [dash.dependencies.Output('graph_students_pass', 'figure'),
     dash.dependencies.Output('graph_sub', 'figure'),
     dash.dependencies.Output('graph_pass', 'figure'),
     dash.dependencies.Output('graph_vle', 'figure'),
     dash.dependencies.Output('graph_std', 'figure'),
     dash.dependencies.Output('pred_div', 'children'),
     dash.dependencies.Output('tma_div', 'children')],
    [dash.dependencies.Input('code_module', 'value'),
     dash.dependencies.Input('code_pres', 'value'),
     dash.dependencies.Input('pass_fail', 'value'),
     dash.dependencies.Input('activity_type', 'value'),
     dash.dependencies.Input('id_student', 'value')]) 

def update_figure(code_module, code_pres, pass_fail, act_type, id_student):
                                     
    
    # Part A
    df_filter = df_stu[df_stu.pred_string == pass_fail]
    df_result = df_filter[(df_filter['code_module'] == code_module) & 
                                            (df_filter['code_presentation'] == code_pres)]
    
    # Table: list of students pass: 
    fig_students = go.Table(
        header=dict(values=['id_student', 'pred_proba', 'prediction (pass/fail)']),
        cells=dict(values=[df_result['id_student'], 
                           df_result['pred_proba'].map('{:,.3f}'.format),
                           df_result['pred_string']]))

    data = [fig_students]
    fig_table = go.Figure(data)
    
    
    
    # Part B
    df_filter = df[np.array(df['code_module'] == code_module) & 
                   np.array(df['code_presentation'] == code_pres)]
    
    if df_filter.empty:
        fig_sub, fig_pass = {}, {}
    
    else:
        sub_data = df_filter.iloc[0,2:5].to_dict()
        data = [d for d in sub_data.items()]
        df_sub = pd.DataFrame(data, columns=['TMA', 'submission_rate'])

        pass_data = df_filter.iloc[0,5:8].to_dict()
        data = [d for d in pass_data.items()]
        df_pass = pd.DataFrame(data, columns=['TMA', 'pass_rate'])

        # Plot submission rate and pass rate
        fig_sub = (px.bar(df_sub, x = 'TMA',
                            y = 'submission_rate',
                            title="Submission Rate"))

        fig_pass = (px.bar(df_pass, x = 'TMA',
                            y = 'pass_rate',
                            title="Pass Rate"))

    
    
    # Part C
    df_filter = vle_grp[np.array(vle_grp['code_module'] == code_module) & 
                   np.array(vle_grp['code_presentation'] == code_pres) &
                   np.array(vle_grp['activity_type'] == act_type)]
    
    if df_filter.empty:
        fig_vle = {}
    else:
        # Plot submission rate and pass rate
        fig_vle = (px.bar(df_filter, x = 'week', y = 'sum_click'))    
    
    
    
    # Part D
    df_filter = table[np.array(table['code_module'] == code_module) & 
                   np.array(table['code_presentation'] == code_pres) &
                   np.array(table['id_student'] == id_student)]
    
    if df_filter.empty:
        fig_last, pass_fail, tma1_score = {}, '', ''

    else:
        # Plot submission rate and pass rate
        fig_last = px.bar(df_filter, x = 'week', y = 'sum_click')

        df_temp = df_filter[['code_module', 'code_presentation', 'id_student', 'TMA1', 'pred']].drop_duplicates()

        if df_temp.iloc[0]['pred'] == 0:
            pass_fail = html.Div([html.Span("Prediction: "), html.Span("FAIL", style={'color':'red'})])
        else:
            pass_fail = html.Div([html.Span("Prediction: "), html.Span("PASS", style={'color':'green'})])

        tma1_score = html.Div("TMA1 weighted score = " + str(df_temp.iloc[0]['TMA1']), 
                                          style={'margin-left':'auto', 'margin-right':'20px', 'text-align': 'right'})

    
    
    return [fig_table, fig_sub, fig_pass, fig_vle, fig_last, pass_fail, tma1_score]
    
# Un-comment to run the app
app.run_server(mode="inline", port =8056)